# ACTIVIDAD 2:
PRÁCTICA DE CLASIFICACIÓN DE TEXTOS.

## Objetivo

El objetivo de esta práctica es realizar una clasificación de una serie de opiniones sobre un hotel, que están recogidas en el fichero hotel.csv. Este fichero contiene dos columnas. La primera, bajo el título text, contiene las opiniones a clasificar, mientras que la segunda, bajo el título label, contiene la puntuación otorgada. Las opiniones se agrupan según su calificación en un valor 5 y otro valor 3.

## Guion de la actividad

1. Lea el contenido del fichero csv en un DataFrame. Se sugiere utilizar la función pandas.read_csv. Atención a la codificación de los datos entrantes.


In [1]:
## Importaciones

import pandas as pd
import numpy as np
import nltk, re, pprint
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import re
import nltk
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("../Actividades/hotel.csv")

In [3]:
data

,text,label
0,Es un gran hotel; el mejor de Asunción. Buenas...,3
1,hola. no suelo criticar jamas lo que paso pero...,3
2,Escogi meses antes de mi boda una habitacion p...,3
3,Voy a se Lo mas equitativo posible; porque soy...,3
4,Esta es una experiencia de septiembre de 2016;...,3
...,...,...
195,Excelente atención; instalaciones; comodidad; ...,5
196,Realmente disfrutamos en pareja por 2 noches d...,5
197,Muy buena excelente; muy buena ubicación; cerc...,5
198,Pasé con mi familia la celebración del Año Nue...,5


Aqui podemos observar como el dataset esta conformado por dos columnas, una con el texto en español donde cada comentario va separado por punto y coma y, luego, va el label con la nota.

2. Realice el pre-procesamiento que considere necesario. Puede utilizar funciones de la librería NLTK o spaCy, a su voluntad. Recomendamos una escritura modular del código, para poder hacer pruebas posteriormente, viendo si se obtienen mejores resultados al utilizar stop-words, al realizar una extracción de formas canónicas, etc.

In [4]:
    # Ponemos los textos en una estructura de datos separada para poder pasarle nuestra funcion de preprocesamiento
corpus = data.iloc[:,0]
categs = data.iloc[:,-1] # extract column with sentiment

In [5]:
####
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('spanish')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [6]:
norm_corpus = normalize_corpus(corpus)
norm_corpus

array(['gran hotel mejor asuncin buenas habitaciones gente maravillosa servicio excelente comida buena cargaron doble estadia expedia hotel hacer checkout llamar varias veces telfono horas devolvieran dinero nico malo gimnasio anuncian gimnasio solo pard caminadoras ojal mejores equipos',
       'hola suelo criticar jamas paso vez animo hacerlo segunda estancia mala experiencia hotel primera probarlo cautivo concepto ecologico hotel segunda forzada tenia lugar alojarmeprimera cosa hotel supone ecologicamente sustentable puede descargar litros litros agua descarga asi lluvia supuestamentepuede tener tantos accesorios banio plastico absolutamente biodegrable menos cumple conceptos sustentabilidad tan buscados ultimamente puede pagar u suite presidencial salir ducha caminar metros mojados busqueda tesoro llamese toallon corriendo riesgo resvalarse hacer ultimo viaje si atencion personalizada noche anterior habitcion simple previamete habia dicho podia mudar suite presidencial mediodia par

3. Divida el conjunto de documentos en un subconjunto de entrenamiento y otro de evaluación.

In [7]:
# División mediante train_test_split. Test de 25%

from sklearn.model_selection import train_test_split
docs_train, docs_test, categs_train, categs_test = train_test_split(norm_corpus, categs, test_size = 0.25, 
                                                                    random_state = 0)

4. Convierta el corpus de documentos en una matriz TF-idf. Lo más cómodo es utilizar el vectorizador TfidfVectorizer, que forma parte de sklearn. ¿Tiene influencia en el resultado final el número máximo de features a utilizar?

In [8]:
# Create feature vectors
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(docs_train)
test_vectors = vectorizer.transform(docs_test)

Hacemos un print del vocabulario que ha aprendido el corpus

In [9]:
print(vectorizer.vocabulary_)

{'hotel': 82, 'mision': 101, 'siempre': 138, 'servicio': 134, 'personal': 115, 'super': 140, 'amable': 5, 'desayuno': 51, 'buen': 21, 'restaurante': 130, 'buena': 22, 'piscina': 117, 'nico': 106, 'gimnasio': 69, 'gran': 70, 'confortable': 44, 'mejor': 97, 'ubicacin': 153, 'gente': 68, 'asuncin': 10, 'comida': 38, 'excelente': 63, 'terraza': 146, 'tranquilo': 151, 'habitaciones': 77, 'general': 67, 'ideal': 84, 'bueno': 24, 'buenos': 25, 'vez': 159, 'am': 3, 'ms': 102, 'tarde': 143, 'trabajo': 150, 'hermoso': 81, 'calidez': 28, 'habitacion': 76, 'hacen': 79, 'comodo': 42, 'restaurant': 129, 'tambien': 141, 'ambiente': 7, 'impecable': 86, 'cama': 29, 'comodas': 40, 'altamente': 2, 'familia': 65, 'mas': 96, 'ser': 133, 'cada': 26, 'mejorar': 98, 'parte': 111, 'cmodas': 36, 'limpieza': 90, 'bien': 18, 'lugar': 94, 'cosa': 46, 'trato': 152, 'atentos': 15, 'variado': 156, 'solo': 139, 'pequeo': 113, 'poca': 118, 'agradable': 1, 'atencion': 13, 'todas': 149, 'instalaciones': 88, 'habitacin': 

Imprimo los datos vectorizados para ver cómo se ven

In [10]:
print(train_vectors)

  (0, 69)	0.3063646977399018
  (0, 106)	0.32702307270222525
  (0, 117)	0.3439021940650491
  (0, 22)	0.2067369389096589
  (0, 130)	0.26679241934919967
  (0, 21)	0.22277823684260212
  (0, 51)	0.1691994425845117
  (0, 5)	0.30038979521046955
  (0, 140)	0.3350784865434649
  (0, 115)	0.17204824757666357
  (0, 134)	0.19490227146891975
  (0, 138)	0.2752144826089034
  (0, 101)	0.353656350193981
  (0, 82)	0.133551376270818
  (1, 151)	0.36548675245158657
  (1, 146)	0.31305898329659787
  (1, 63)	0.15808059185258186
  (1, 38)	0.2531051575057866
  (1, 10)	0.2455791008699641
  (1, 68)	0.3304160125891931
  (1, 153)	0.21050836100757056
  (1, 97)	0.42854428381610343
  (1, 44)	0.35215347596558455
  (1, 70)	0.3304160125891931
  (1, 134)	0.18209437310545695
  :	:
  (147, 134)	0.17026872850735572
  (148, 100)	0.3548921784958716
  (148, 62)	0.36832914806043576
  (148, 52)	0.3548921784958716
  (148, 4)	0.28845813685060934
  (148, 12)	0.2074325688855664
  (148, 118)	0.36832914806043576
  (148, 156)	0.323801622

5. Llegados a este punto, realice modelos de entrenamiento al menos con algoritmos de clasificador bayesiano ingenuo y máquinas SVM. Obtenga resultados de precisión de la clasificación, así como las matrices de confusión para ambos modelos.

In [11]:
# Entrenamiento del clasificador Naive Bayes

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(train_vectors, categs_train)

MultinomialNB()

In [12]:
# Predicción del set de test

categs_pred = clf.predict(test_vectors)

In [13]:
# Confusion Matrix

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(categs_test, categs_pred)
cm

array([[21,  4],
       [12, 13]])

In [14]:
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(categs_pred, categs_test)*100)

Naive Bayes Accuracy Score ->  68.0


In [15]:
# Entrenamiento con Support Vector Machine

from sklearn import svm
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')

classifier_linear.fit(train_vectors, categs_train)

prediction_linear = classifier_linear.predict(test_vectors)


In [16]:
## Matriz de confusión

cm = confusion_matrix(categs_test, prediction_linear)
cm

array([[20,  5],
       [10, 15]])

In [17]:
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(prediction_linear, categs_test)*100)

SVM Accuracy Score ->  70.0


Podemos observar que el SVM machine predice mejor que el modelo Naive Bayes ya que consigue mejor accuracy

6. Comente los resultados obtenidos. ¿Qué factores influyen? ¿Los resultados obtenidos son los esperados inicialmente? ¿A qué se deben estos resultados? Piense en la calidad del conjunto de datos con los que está trabajando

Luego de haber probado los algoritmos de clasifiacacion Naive Bayes y el Support Vector Machine, podemos observar que utilizando el accuracy como metrica para comparar los resultados, el modelo que mejor predice es el SVM.

Inicialmente me esperaba que el SVM pueda conseguir mejores resultados ya que su algoritmo esta diseñado para separar variables que no tienen relaciones lineales. Esto se logra a traves de los kernel que le pasamos al modelo.

El preprocesamiento de los datos juega un factor fundamental en estos resultados como en cualquier proceso de minería de datos. Al transformar los datos sin procesar (usualmente incompletos, inconsistentes o que carecen de de ciertos comportamientos o tendencias) en un formato comprensible para los modelos de PNL, nos ayuda a obtener mejores resultados con los algoritmos de clasificación.